In [1]:
import os
import numpy as np
import pandas as pd
import jinja2 as jj


In [2]:
def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product([mklbl('A', 4),
                                      mklbl('B', 2),
                                      mklbl('C', 4),
                                      mklbl('D', 2)],
                                    names=['RowIdx-1', 'RowIdx-2', 'RowIdx-3', 'RowIdx-4'])
index  =['-'.join(col).strip() for col in miindex.values]
micolumns = pd.MultiIndex.from_tuples([('a', 'foo', 'zap'),
                                       ('a', 'foo', 'zip'),
                                       ('a', 'bar', 'zap'),
                                       ('a', 'bar', 'zip'),
                                       ('b', 'foo', 'zap'),
                                       ('b', 'foo', 'zep'),
                                       ('b', 'bah', 'zep'),
                                       ('b', 'bah', 'zyp'),
                                       ('b', 'bah', 'zap'),
                                      ],
                                      names=['ColIdx-{}'.format(i) for i in range(1, 4)])
cols  =['-'.join(col).strip() for col in micolumns.values]
data = np.arange(len(miindex) * len(micolumns), dtype=np.float).reshape((len(miindex),len(micolumns)))
data = data.tolist()


dfrc = pd.DataFrame(data, index=miindex, columns=micolumns).sort_index().sort_index(axis=1)

dfr  = pd.DataFrame(data, index=miindex, columns=cols).sort_index().sort_index(axis=1)
dfr.columns.name = 'UniqueCol'

dfc  = pd.DataFrame(data, index=index, columns=micolumns).sort_index().sort_index(axis=1)
dfc.index.name = 'UniqueRow'

df  = pd.DataFrame(data, index=index, columns=cols).sort_index()
df.index.name = 'UniqueRow'
df.columns.name = 'UniqueCol'

dfrc.info()
dfr.info()
dfc.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 64 entries, ('A0', 'B0', 'C0', 'D0') to ('A3', 'B1', 'C3', 'D1')
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (a, bar, zap)  64 non-null     float64
 1   (a, bar, zip)  64 non-null     float64
 2   (a, foo, zap)  64 non-null     float64
 3   (a, foo, zip)  64 non-null     float64
 4   (b, bah, zap)  64 non-null     float64
 5   (b, bah, zep)  64 non-null     float64
 6   (b, bah, zyp)  64 non-null     float64
 7   (b, foo, zap)  64 non-null     float64
 8   (b, foo, zep)  64 non-null     float64
dtypes: float64(9)
memory usage: 5.6+ KB
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 64 entries, ('A0', 'B0', 'C0', 'D0') to ('A3', 'B1', 'C3', 'D1')
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   a-bar-zap  64 non-null     float64
 1   a-bar-zip  64 non-null     float64
 2   a-foo-zap  64 no

/var/folders/wq/sl9cqpsj3w76ywvkf3nbp24c0000gn/T/ipykernel_53910/2942104012.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = np.arange(len(miindex) * len(micolumns), dtype=np.float).reshape((len(miindex),len(micolumns)))


In [3]:
dfrc.head()

ColIdx-1                                a                       b              \
ColIdx-2                              bar         foo         bah               
ColIdx-3                              zap   zip   zap   zip   zap   zep   zyp   
RowIdx-1 RowIdx-2 RowIdx-3 RowIdx-4                                             
A0       B0       C0       D0         2.0   3.0   0.0   1.0   8.0   6.0   7.0   
                           D1        11.0  12.0   9.0  10.0  17.0  15.0  16.0   
                  C1       D0        20.0  21.0  18.0  19.0  26.0  24.0  25.0   
                           D1        29.0  30.0  27.0  28.0  35.0  33.0  34.0   
                  C2       D0        38.0  39.0  36.0  37.0  44.0  42.0  43.0   

ColIdx-1                                         
ColIdx-2                              foo        
ColIdx-3                              zap   zep  
RowIdx-1 RowIdx-2 RowIdx-3 RowIdx-4              
A0       B0       C0       D0         4.0   5.0  
                           D1        13.0  14.0  
                  C1       D0        22.0  23.0  
                           D1        31.0  32.0  
                  C2       D0        40.0  41.0

## Save df html
Must:
+ use notebook.css
+ wrap dataframe.html in specific classes - like in notebook
Result can be iframed in any doc


In [4]:
%%writefile templates/index.tpl.html

<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <title>dataframe</title>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://cdn.jupyter.org/notebook/5.6.0/style/style.min.css">
</head>

<body>


    <div class="output_are">
        <div class="output_subarea output_html rendered_html output_result">

            __$data.df_html$__

        </div>
    </div>

</body>

</html>

Overwriting templates/index.tpl.html


In [5]:
dir_template = 'templates'
dir_dump = 'dump'

loader = jj.FileSystemLoader(dir_template)
env = jj.Environment(loader=loader,
                     variable_start_string='__$',
                     variable_end_string='$__',
                     block_start_string='{-%',
                     block_end_string='%-}'
                     )

template = env.get_template('index.tpl.html')
# data = {'df_html': dfrc.to_html()}
data = {'df_html': dfrc.head(10).to_html()}
content = template.render(data=data)

if not os.path.exists(dir_dump):
    os.makedirs(dir_dump)

path = os.path.join(dir_dump, 'index.html')
with open(path, 'w') as f:
    f.write(content)
    print('file {} saved to disk'.format(path))


file dump/index.html saved to disk


In [ ]:
!cd dump && python -m http.server 8080